In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [2]:
data=pd.read_csv("../datasets/titanic/train.csv")

In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
final=data.drop(["PassengerId","Name","Ticket","Fare","Cabin","Embarked"],axis=1)

In [5]:
final.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,male,22.0,1,0
1,1,1,female,38.0,1,0
2,1,3,female,26.0,0,0
3,1,1,female,35.0,1,0
4,0,3,male,35.0,0,0


In [6]:
final["Sex"]=LabelEncoder().fit_transform(final["Sex"])

In [7]:
final.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,1,22.0,1,0
1,1,1,0,38.0,1,0
2,1,3,0,26.0,0,0
3,1,1,0,35.0,1,0
4,0,3,1,35.0,0,0


In [8]:
final.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000
mean,0.383838,2.308642,0.647587,29.699118,0.523008,0.381594
std,0.486592,0.836071,0.477990,14.526497,1.102743,0.806057
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,20.125000,0.000000,0.000000
50%,0.000000,3.000000,1.000000,28.000000,0.000000,0.000000
75%,1.000000,3.000000,1.000000,38.000000,1.000000,0.000000
max,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000


In [9]:
processed=final.dropna()

In [10]:
processed.shape

(714, 6)

In [11]:
processed1=processed.groupby(["Pclass","Sex"]).mean()[["Age"]].reset_index()

In [12]:
processed1

,Pclass,Sex,Age
0,1,0,34.611765
1,1,1,41.281386
2,2,0,28.722973
3,2,1,30.740707
4,3,0,21.750000
5,3,1,26.507589


In [13]:
unprocessed=final.loc[final["Age"].isna()]

In [14]:
unprocessed.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
5,0,3,1,NaN,0,0
17,1,2,1,NaN,0,0
19,1,3,0,NaN,0,0
26,0,3,1,NaN,0,0
28,1,3,0,NaN,0,0


In [15]:
def agefill(row):
    row["Age"]=processed1.loc[(row["Pclass"]==processed1["Pclass"])&(row["Sex"]==processed1["Sex"])].describe().loc["mean","Age"]
    return row

In [16]:
processed3=unprocessed.apply(agefill,axis=1)

In [17]:
processed3.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
5,0.0,3.0,1.0,26.507589,0.0,0.0
17,1.0,2.0,1.0,30.740707,0.0,0.0
19,1.0,3.0,0.0,21.750000,0.0,0.0
26,0.0,3.0,1.0,26.507589,0.0,0.0
28,1.0,3.0,0.0,21.750000,0.0,0.0


In [18]:
final1=pd.concat([processed,processed3])

In [19]:
final1.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0.0,3.0,1.0,22.0,1.0,0.0
1,1.0,1.0,0.0,38.0,1.0,0.0
2,1.0,3.0,0.0,26.0,0.0,0.0
3,1.0,1.0,0.0,35.0,1.0,0.0
4,0.0,3.0,1.0,35.0,0.0,0.0


In [20]:
X=final1.iloc[:,1:]
y=final1.iloc[:,0]

In [21]:
model=DecisionTreeClassifier()

In [22]:
model.fit(X,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [23]:
model.score(X,y)

0.920314253647587

In [24]:
def entropy(data):
    m=len(data)
    val,count=np.unique(data,return_counts=True)
    p=count/m
    return np.sum(-p*np.log(p))

In [25]:
def info_gain(X,y,label):
    x_col=X[label]
    pivot=np.mean(x_col)
    
    left  = x_col<pivot
    right = x_col>=pivot
 
    y_left,y_right=y.loc[left],y.loc[right]
    
    ent=entropy(y)
    ent_l=entropy(y_left)
    ent_r=entropy(y_right)
    
    prob_l=len(y_left)/len(y)
    prob_r=1-prob_l
    
    return ent  -prob_l*ent_l -prob_r*ent_r

In [26]:
for col in X.columns:
    print(col,info_gain(X,y,col))

Pclass 0.05253613916173083
Sex 0.15087048925218172
Age 0.0007663481843047659
SibSp 0.006643498134917131
Parch 0.010661126611803073


In [27]:
class Node:
    def __init__(self,label=None,pivot=None,result=None):
        self.label=label
        self.pivot=pivot
        self.result=result
        
        self.left=None
        self.right=None
        
        
    def __repr__(self):
        
        if self.label == None:
            return str(self.result > .5)
        else:
            return str(self.label + " : " + str(self.pivot))

In [52]:
class coutom:
    
    def __init__(self,maxiter=10):
        self.maxiter=maxiter
    
    def fit(self,X,y):
        self.root=self.rec_fit(X,y,maxx=self.maxiter)
        
    def rec_fit(self,X,y,maxx):
        if maxx ==0:
            return Node(result=y.mean())
        
        gains=[]
        
        for col in X.columns:
            gains.append((info_gain(X,y,col),col))
        
        selected_gain,selected_col=sorted(gains)[-1]
        
        if selected_gain<=0:
            return Node(result=y.mean())
            
        pivot=X[selected_col].mean()
        
        left=X[selected_col]<pivot
        right=X[selected_col]>=pivot
        
        X_left , X_right = X.loc[left],X.loc[right]
        y_left ,y_right =y.loc[left] ,y.loc[right]
        
        node=Node(label=selected_col,pivot=pivot)
        
        node.left=self.rec_fit(X_left,y_left,maxx-1)
        node.right=self.rec_fit(X_right,y_right,maxx-1)
        
        return node
    
    def display(self):
        self.display_rec(self.root)
        
    def display_rec(self, node, indent=""):
        if node == None:
            return
        
        print(indent, node)
        self.display_rec(node.left, indent + "\t")
        self.display_rec(node.right, indent + "\t")
        
    def predicted_point(self,row):
        self.predicted_rec(self.root,row)
        
    def predicted_rec(self,node,row):
        if node.label==None:
            return
        print(row[node.label])
        if row[node.label]<node.pivot:
            return self.predicted_rec(node.left,row)
        else:
            return self.predicted_rec(node.right,row)
        
    def predicted(self,X):
        err=[]
        for index,row in X.iterrows():
            err.append(self.predicted_point(row))
        return (np.array(err) >= .5).astype(int)

In [53]:
a=coutom()

In [54]:
a.fit(X,y)

In [55]:
a.display()

 Sex : 0.6475869809203143
	 Pclass : 2.159235668789809
		 Pclass : 1.4470588235294117
			 Parch : 0.4574468085106383
				 Age : 36.07647058823528
					 True
					 Age : 49.08695652173913
						 True
						 SibSp : 0.7272727272727273
							 Age : 57.0
								 False
								 True
							 True
				 Parch : 1.4333333333333333
					 True
					 SibSp : 1.0769230769230769
						 SibSp : 0.4444444444444444
							 True
							 False
						 True
			 Parch : 0.6052631578947368
				 Age : 31.909909909909906
					 SibSp : 0.46153846153846156
						 Age : 25.15287162162162
							 Age : 20.714285714285715
								 True
								 Age : 23.0
									 True
									 Age : 23.666666666666668
										 True
										 False
							 True
						 Age : 25.5
							 True
							 Age : 28.285714285714285
								 Age : 27.5
									 False
									 True
								 True
					 Age : 40.973684210526315
						 Age : 35.77272727272727
							 True
							 Age : 37.666666666666664
								 True
								 Age : 39

In [56]:
a.predicted(X[:5])

1.0
3.0
0.0
22.0
1.0
22.0
1.0
0.0
1.0
1.0
0.0
38.0
38.0
0.0
3.0
0.0
26.0
0.0
26.0
26.0
26.0
26.0
0.0
1.0
1.0
0.0
35.0
1.0
3.0
0.0
35.0
0.0
35.0
35.0


TypeError: '>=' not supported between instances of 'NoneType' and 'float'